# Multi Attributes Fractal Dimension

## Exploratory Data Analysis

Summarize about "SeisLab3.02" package. You can download from Mathworks [here](https://www.mathworks.com/matlabcentral/fileexchange/53109-seislab-3-02)

In [1]:
% Folder with 'SeisLab3.02' package
addpath(genpath('SeisLab3.02'))

% Folder with 'Segymat' package
addpath(genpath('Segymat'))

% Folder with data
addpath(genpath('Data'))

% Folder with Assets
addpath(genpath('Assets'))

In [2]:
tic
data = read_segy_file('Data/Inline 900-903/instantaneous_amplitude_IL_900-903.segy', ...
                     {'headers', {'iline_no', 189, 4, 'n/a', 'In-line number'}})
toc

Seismic data shifted since header "lag" is not identically zero.
Lag varies from 1300 to 1300

data = 

  struct with fields:

                      type: 'seismic'
                       tag: 'unspecified'
                      name: 'instantaneous_amplitude_IL_900-903 - shifted'
                      from: 'Data\Inline 900-903\instantaneous_amplitude_IL_900-903.segy'
               line_number: 1
               reel_number: 1
         traces_per_record: 1
                  cdp_fold: 1
                     first: 1300
                      last: 3300
                      step: 4
                     units: 'ms'
                    traces: [501x5132 double]
                      null: []
                   history: {2x4 cell}
               header_info: {13x3 cell}
                   headers: [13x5132 double]
    fp_format_of_segy_file: 'ibm'

Elapsed time is 1.273528 seconds.



In [3]:
format shortG
data.traces(1:5, 100)


ans =

       674.98
       639.51
       595.04
       484.41
       321.41




In [4]:
data.header_info


ans =

  13x3 cell array

    {'ds_seqno'  }    {'n/a'}    {'Trace sequence number within line'   }
    {'ffid'      }    {'n/a'}    {'Original Field record number'        }
    {'o_trace_no'}    {'n/a'}    {'Trace sequence number within orig...'}
    {'cdp'       }    {'n/a'}    {'CDP number'                          }
    {'seq_cdp'   }    {'n/a'}    {'Trace sequence number within CDP ...'}
    {'sou_x'     }    {'m'  }    {'X coordinate of source'              }
    {'sou_y'     }    {'m'  }    {'Y coordinate of source'              }
    {'cdp_x'     }    {'n/a'}    {'X-coordinate of CDP'                 }
    {'cdp_y'     }    {'n/a'}    {'Y-coordinate of CDP'                 }
    {'xline_no'  }    {'n/a'}    {'Cross-line number'                   }
    {'trc_type'  }    {'n/a'}    {'Trace type (1=live,2=dead,3=dummy...'}
    {'lag'       }    {'ms' }    {'Lag time between shot and recordi...'}
    {'iline_no'  }    {'n/a'}    {'In-line number'                      }




## Input data

Just in case you need to change the [format](https://www.mathworks.com/help/matlab/ref/format.html) to display data.

In [5]:
% format longG
format compact

In [6]:
% Definition of time window, above and below the horizon of interest.
TIME_ABOVE_HORIZON = 0;
TIME_BELOW_HORIZON = 20;

% Array of strings containing the names of the seismic attribute files.
files = {'Data/Inline 900-903/seismic_amplitude_IL_900-903.segy', ...
         'Data/Inline 900-903/instantaneous_amplitude_IL_900-903.segy', ...
         'Data/Inline 900-903/instantaneous_frecuency_IL_900-903.segy'};
         
% Array of strings containing the titles of the input seismic attributes.
titlesFiles = {'Seismic amplitude', 'Instantaneous amplitude', 'Instantaneous frequency'};

In [7]:
% Definition of the weights for each attributes. It must be the same quantity as the input attributes and their sum must be
% equal to 1. These are used to control the contribution of each seismic attribute in the calculation of Fractal Dimension.
ALPHA = [0.33, 0.33, 0.33];

% File to save the calculation of Fractal Dimension.
fileFractalDim = {'Data/EM_MER-U2_Fractal_Dimension.prn'};

% Number of traces to perform Quality Control (QC )of the calculations.
NUMTRACESQC = 5;

% Minimum and Maximum values of input seismic data. Extracted from Textual Header. [[min, max]; [min, max]; [min, max]]
MINMAX = [[-7469.42, 6192.14]; [0.0, 5228.60]; [0.0, 125.0]];

## Load data 

### Surface Data

In [8]:
% Opening of the time surface of interest. The format must be X - Y - Time. Otherwise, it would generate an error.
% X-Y are coordinates.
assy = fopen('Data/EM_MER-U2.prn');

switch assy
    case -1
        errordlg('The surface file is not in the current directory', 'Error!');
    otherwise
        try
            reg = fscanf(assy, '%10f');
            surfaceData = reshape(reg,3,[])';
        catch id
            errordlg('The file does not have the format X-Y-Time', 'Error!'); 
        end 
end

In [9]:
size(surfaceData)

ans =
      504175           3



In [10]:
% First 5 rows of surface data
format longG
surfaceData(1:5, :)

ans =
                    368425                   1018414                 2335.5974
                    368425                   1018564                 2332.6064
                    368425                   1018939                 2360.4221
                    368425                   1019014                 2350.2068
                    368425                   1019464                    2396.7



### Seismic data

In [11]:
% Number of attributes to use in the algorithm
nAttributes = size(files, 2);

In [12]:
% Raise error dialog if the sum of the weights is different from one, or they differ from the number of input attributes.

if (sum(ALPHA) < 0.99)
    errordlg('The sum of weights are not equal to 1', 'Error!'); 
    
elseif (nAttributes ~= size(ALPHA,2))
    errordlg('The number of weights is not equal to the input attributes.', 'Error!'); 
    
end

The following data must be the same for each seismic volume.
- dtSample: Sampling interval (microseconds).
- nSamples: Number of samples.
- nTraces: Number of traces.

In [13]:
% If there was selected only one seismic volume, no verification need it.

if nAttributes >= 2
    [dtSample, nSamples, nTraces] = checkSegyData(files);
else
    [segyHeader] = ReadSegyHeader(files{1});
    dtSample= segyHeader.dt/1000;
    nSamples= segyHeader.ns;
    nTraces = segyHeader.ntraces;
end

In [14]:
fprintf('Sampling interval: %d ms.   Number of samples: %d.   Number of traces: %d.\n', dtSample, nSamples, nTraces); 

Sampling interval: 4 ms.   Number of samples: 501.   Number of traces: 5132.



## Data preprocessing

### Time column

In [15]:
% Reade segy header
[segyHeader] = ReadSegyHeader(files{1});

% Read segy trace header
% 'SkipData': Read only the header values (Data will return empty)
[~ , segyTrace1Header] = ReadSegy(files{1}, 'traces', 1, 'SkipData', 1);

In [16]:
% From "segyHeader" read time column and sampling interval. Multiply and divide by 1000 to get miliseconds.
% From "segyTraceHeader" read "Delay Recording Time".
timeColumn = segyHeader.time*1000 - segyHeader.dt/1000 + segyTrace1Header.DelayRecordingTime;

In [17]:
% k1 and k2. Number of samples to extract seismic data for Fractal Dimension analysis.
% It depends on the time window previously chosen.
nTop = ceil(TIME_ABOVE_HORIZON/dtSample);
nBottom = ceil(TIME_BELOW_HORIZON/dtSample);

### Seismic data

X-Y coordinates of all traces in the seismic volume. These coordinates will be the same in all input seismic volumes, because the sampling interval, the number of samples and the number of traces are the same. Also, they are divided by 100 to bring them to metric scale. 

``` matlab
coordinatesSeismicTraces = zeros(nTraces, 2);

% <B = A.'> is equal to <B = transpose(A)>
coordinatesSeismicTraces(:,1) = ReadSegyTraceHeaderValue(files{1}, 'key', 'cdpX').';
coordinatesSeismicTraces(:,2) = ReadSegyTraceHeaderValue(files{1}, 'key', 'cdpY').';

% Divide each coordinate by 100
coordinatesSeismicTraces = round(traceCoordinates./100);
```

It is very common seismic data with a constant number of seismic traces per Inline (so it is in these data). However, trace sequence line in segy file starts at 1 for each Inline, it does not follow a linear sequence. This sequence is calculated here, which will allow a much faster reading of data from a large file.

In [79]:
% Note: If this line is too slow reading all seismic volume, I suggest read coordinates using prior text code, calculate the size 
% of "coordinatesSeismicTraces(:,1)" and then contruct a linspace between 1 and this size. If the seismic file has an Inlines complete, 
% the mod between this size an the number of traces by seismic line will be zero.
coordinatesSeismicTraces = traceNumbers(files{1}, nTraces);

Select X-Y coordinates and the number of seismic traces where surface data was interpolated. 

In [20]:
selectedCoordinates = selectEqualCoord(nTraces, surfaceData, coordinatesSeismicTraces);

In [21]:
fprintf('Number of traces in seismic data is : %d.\nNumber of traces selected to Fractal Dimension Calculations: %d', ...
        size(coordinatesSeismicTraces, 1), size(selectedCoordinates, 1)); 

Number of traces in seismic data is : 5132.
Number of traces selected to Fractal Dimension Calculations: 1312


### Surface data

Select surface data with coordinates equal to seismic data.

In [22]:
selectedSurface = selectEqualCoord(nTraces, selectedCoordinates, surfaceData);

### Preallocating matrices

In [23]:
% Number of samples selected to Fractal dimension calculations.
nTracesSelected = size(selectedCoordinates, 1);

In [24]:
% This matrix will store the results of Fractal Dimension calculations.
results = zeros(nTracesSelected, 3);

% This vector will store the values ??of R2 in order to make a histogram.
allR2 = zeros(nTracesSelected, 1);

Definition of random numbers between 1 and the number of traces in seismic volumes. This will represent random traces, in order to perform Quality Control of Fractal Dimension calculations. In order to generate repeatable random numbers, we use:

``` python
rng(3, 'twister');
```

More information in this [link](https://www.mathworks.com/help/matlab/math/generate-random-numbers-that-are-repeatable.html).

In [25]:
rng(3, 'twister');
tracesQC = round(rand(1, NUMTRACESQC) * nTraces);

In [26]:
tracesQC

tracesQC =
        2827        3634        1493        2622        4583



Description of the matrices:
1. "*seismicTraces*": Save seismic attributes traces.
2. "*params*": Save the number of traces, slope, Fractal Dimension and the coefficient of determination R2.
3. "*coordinatesQC*": Save traces coordinates for quality control.
4. "*timeAnalysis*": Time interval analysis.
5. "*divisorLength*": Save "dividers" and "length". The length of both vectors is unknown, so pre-allocation is made with a single column.
6. "*traceNumbers*": number of the traces used to calculations (the coordinates of the surface is equal to the coordinate of the seismic trace).

In [27]:
seismicTraces = zeros(nTop+nBottom+1, nAttributes, NUMTRACESQC);

params = zeros(NUMTRACESQC, 4);

coordinatesQC = zeros(NUMTRACESQC, 2);

timeAnalysis =  zeros(nTop+nBottom+1, NUMTRACESQC);

dividerLength = zeros(2, 1, NUMTRACESQC);

% traceNumbers = zeros(1, nTracesSelected) ;

## Multi-attributes Fractal Dimension Algorithm

In [28]:
% Initialize waitbar
wb = waitbar(0, 'Analyzing data');

% traceNum = index that locates the trace number read.
traceNum = 1;

% conditionQC: Quality Control Condition. Sets the selection of
% the random trace to save its corresponding data.
conditionQC = false;

% traceQC: index that indicates the trace number to QC.
traceQC = 1;

In [29]:
iPoint = 1;

In [30]:
fprintf('Seismic Trace number: %d', selectedCoordinates(iPoint, 3));

Seismic Trace number: 38


In [31]:
% Plot waitbar
waitbar(iPoint/nTracesSelected, wb)

% Read trace number selected in seismic data.
seismicData = zeros(nSamples, nAttributes);
for iAttribute = 1 : nAttributes
    % Read Segy data of each input
    [Data, ~] = ReadSegy(files{iAttribute}, 'traces', selectedCoordinates(iPoint, 3));
    seismicData(:, iAttribute) = Data;
end

In [32]:
% First 5 values of seismic attributes data
format shortG
seismicData(1:5, :)

ans =
       290.72       306.46       29.412
       363.95       399.81       28.922
       183.87        470.5       28.922
      -172.79       516.52       28.431
      -466.83       530.66       27.941



In [33]:
% Check if any of the seismic attributes data are empty. If this case,
% it will not calculate Fractal Dimension.  
isEmptyData= any(all(seismicData==0));

In [34]:
% Get time sample of surface data. "floor": Round towards -Inf.
timeSample = floor(selectedSurface(iPoint,3));

% Check if "timeSample" is a multiple of sampling interval of seismic data (4 miliseconds in this case).
% Otherwise, take the next higher multiple.
modulus = mod(timeSample, dtSample);
while modulus~=0
    timeSample = timeSample - 1;
    modulus = mod(timeSample, dtSample);
end

In [35]:
% Find the position of "timeSample" in "timeColumn" vector.
indexTimeSample = find(timeSample == timeColumn);

In [36]:
% Select seismic attributes data between time window previously defined. This will be the data under analysis.
analyzedData = zeros(nTop + nBottom + 1, nAttributes);

for i = 1 : nAttributes
    analyzedData(:,i) = seismicData(indexTimeSample - nTop : indexTimeSample + nBottom, i);
end

In [37]:
% Normalization of data
normalizedData = minMaxScaler(analyzedData, MINMAX);

In [38]:
% Weighed normalized data
normalizedData = normalizedData .* ALPHA;

In [39]:
% Fractal Dimension calculation
[r,L,P,R2] = divplot2_EM(normalizedData)

r =
    0.0060724     0.012145     0.018217      0.02429
L =
     0.050468     0.047194     0.037538      0.04361
P =
     -0.15647
R2 =
      0.54569

